In [ ]:
#Who is the best basketball player of all time? I will be looking at diving into
#the analytics of the Goat debate, to decide if Michael Jordan or Lebron James is the 
#greatest basketball player ever. I will be analyzing what factors into that distinction, including
#advanced stats, longevity, winning metrics, and team analytics.
#I used the NBA_api to pull the data

In [61]:
from nba_api.stats.static import players
from nba_api.stats.library.parameters import SeasonAll
from nba_api.stats.endpoints import playergamelog
import pandas as pd 
import plotly.express as px
import plotly



In [63]:
#Get players from dictionary that I pulled from the API.
player_dict = players.get_players()

#Get Lebron's player ID
lebron = [player for player in player_dict if player['full_name'] == 'LeBron James'][0]
lebron_id = lebron['id']

#Get Jordan's Player ID
mj = [player for player in player_dict if player['full_name'] == 'Michael Jordan'][0]
mj_id = mj['id']

#Get gamelogs for each player and combine them into 1 dataframe
lebron_gamelog = playergamelog.PlayerGameLog(player_id=lebron_id, season = SeasonAll.all)
lebron_gamelog_df = lebron_gamelog.get_data_frames()

mj_gamelog = playergamelog.PlayerGameLog(player_id=mj_id, season = SeasonAll.all)
mj_gamelog_df = mj_gamelog.get_data_frames()

combined_df = pd.concat([mj_gamelog_df[0], lebron_gamelog_df[0]],  ignore_index=False)

In [64]:
#Write a csv file of the Combined dataframe
combined_df.to_csv('goat_data.csv', index=False)

In [65]:
#Turn the player dictionary into a dataframe
player_dict = pd.DataFrame(player_dict)
player_dict = player_dict[['id','full_name']]


In [66]:
#Create a Dataframe of each game agregated by player and season, and create a new column called PPG
result_df = combined_df.groupby(['SEASON_ID', 'Player_ID']).agg({'REB': 'sum', 'PTS': 'sum','Game_ID': 'count'}).reset_index()
result_df['Games_Played'] = result_df['Game_ID']
result_df = result_df.drop('Game_ID', axis=1)
result_df['id'] = result_df['Player_ID']
result_df = result_df.drop('Player_ID', axis=1)
result_df['PPG'] = result_df['PTS']/result_df['Games_Played']
result_df = pd.merge(result_df, player_dict, on='id', how='left')
result_df['Season'] = result_df.groupby('full_name')['SEASON_ID'].rank(ascending=False).astype(int)
result_df


,SEASON_ID,REB,PTS,Games_Played,id,PPG,full_name,Season
0,21984,534,2313,82,893,28.207317,Michael Jordan,15
1,21985,64,408,18,893,22.666667,Michael Jordan,14
2,21986,430,3041,82,893,37.085366,Michael Jordan,13
3,21987,449,2868,82,893,34.975610,Michael Jordan,12
4,21988,652,2633,81,893,32.506173,Michael Jordan,11
5,21989,565,2753,82,893,33.573171,Michael Jordan,10
6,21990,492,2580,82,893,31.463415,Michael Jordan,9
7,21991,511,2404,80,893,30.050000,Michael Jordan,8
8,21992,522,2541,78,893,32.576923,Michael Jordan,7
9,21994,117,457,17,893,26.882353,Michael Jordan,6


In [60]:
#Create a Plot showing the trend in Points per game for each season of the player's career
fig = px.line(result_df, x='Season', y='PPG', color='full_name', title='PPG for Each Season of Career',
                 labels={'PPG': 'Points Per Game', 'Season': 'Season', 'full_name':'Name'},
                 hover_data=['full_name'])

fig.show()
plotly.offline.plot(fig, filename='ppg_plot.html')


'ppg_plot.html'